In [1]:
import pandas as pd

In [2]:
from web_tracking.pre_processing.raw_data_processor import RawDataProcessor
from web_tracking.pre_processing.trajectories.binned_non_stationary_trajectory import BinnedNonStationaryTrajectory

# Load raw data

<div class="alert alert-warning">
<b>Warning:</b> Update the data directory path before running.
</div>

In [4]:
data_directory = "../../../../../../data/web_routineness_release_clean/"

In [5]:
urls = pd.read_csv(
    data_directory + "/pre_processed/browsing_with_gap.csv", 
    dtype={'id': int, 'panelist_id': int, 'prev_id': int}, parse_dates=['used_at', 'left_at'])

In [6]:
urls.head()

,id,prev_id,panelist_id,used_at,left_at,active_seconds,gap_seconds,top_level_domain,category_names_top,sub_level_domain,subdomain,category_names_sub,domain,category_names
0,1009076504,0,0,2018-10-01 00:03:29,2018-10-01 00:03:31,2,0,youtube.com,"entertainment,streaming-media",NaN,NaN,NaN,youtube.com,"entertainment,streaming-media"
1,1009076508,1009076504,0,2018-10-01 00:03:31,2018-10-01 00:03:37,6,0,youtube.com,"entertainment,streaming-media",NaN,NaN,NaN,youtube.com,"entertainment,streaming-media"
2,1009076512,1009076508,0,2018-10-01 00:03:37,2018-10-01 00:03:43,6,0,youtube.com,"entertainment,streaming-media",NaN,NaN,NaN,youtube.com,"entertainment,streaming-media"
3,1009076516,1009076512,0,2018-10-01 00:03:43,2018-10-01 00:03:49,6,0,youtube.com,"entertainment,streaming-media",NaN,NaN,NaN,youtube.com,"entertainment,streaming-media"
4,1009076520,1009076516,0,2018-10-01 00:03:49,2018-10-01 00:03:53,4,0,youtube.com,"entertainment,streaming-media",NaN,NaN,NaN,youtube.com,"entertainment,streaming-media"


# Filtering users

In [7]:
RDP = RawDataProcessor(urls)
RDP.create_time_series(uid='id',
                       user='panelist_id',
                       url='url',
                       domain='domain',
                       starts='used_at',
                       active_seconds='active_seconds')

In [8]:
BNST = BinnedNonStationaryTrajectory(RDP.df)
BNST_domain, = BNST.create(features=['domain'], bin_size=60).values()

In [9]:
group_by_user = BNST_domain.groupby('user', as_index=False).agg({'domain': 'count'})
selected_users = group_by_user.query('domain >= 100')[['user']]

In [10]:
len(selected_users)

1455

In [11]:
selected_users.to_csv(data_directory + '/pre_processed/selected_users.csv', index=False)